In [16]:
%%python2
s = r'''
def out10(  self, numberFile, letterFile ):
    number = open( numberFile().path, 'r').read().strip()
    letter = open( letterFile().path, 'r').read().strip()
    with self.output_kw['OUT'].open("w") as f:
        f.write( 25 * (number+letter)+'\n')
    return

'''
import ast
ast.dump(ast.parse(s))
from _ast_util import ast_proj
print ast_proj(s)


FunctionDef(name='out10', body=[Assign(targets=[Name(id='number', ctx=Store())], value=Call(func=Attribute(value=Call(func=Attribute(value=Call(func=Name(id='open', ctx=Load()), args=[Attribute(value=Call(func=Name(id='numberFile', ctx=Load()), args=[], keywords=[], starargs=None, kwargs=None), attr='path', ctx=Load()), Str(s='r')], keywords=[], starargs=None, kwargs=None), attr='read', ctx=Load()), args=[], keywords=[], starargs=None, kwargs=None), attr='strip', ctx=Load()), args=[], keywords=[], starargs=None, kwargs=None)), Assign(targets=[Name(id='letter', ctx=Store())], value=Call(func=Attribute(value=Call(func=Attribute(value=Call(func=Name(id='open', ctx=Load()), args=[Attribute(value=Call(func=Name(id='letterFile', ctx=Load()), args=[], keywords=[], starargs=None, kwargs=None), attr='path', ctx=Load()), Str(s='r')], keywords=[], starargs=None, kwargs=None), attr='read', ctx=Load()), args=[], keywords=[], starargs=None, kwargs=None), attr='strip', ctx=Load()), args=[], keywords=

In [78]:
from pipedata import cached_property
some=[]
import inspect
class a():
    @cached_property
    def f(self):
        b =1
        c = 2
        a= inspect.currentframe().f_lasti
        print (bytes(a))
        import dis
        s= inspect.currentframe().f_code.co_code[a:]
        print (s, dis.dis(s))
#         print a
#         print (bytes(inspect.currentframe().f_lasti,))
# co_names
        return some
print a().f
some.append(123)
print a().f
def g():
#     ...
    pass
g.func_code.co_name

21
          0 LOAD_ATTR           2 (2)
          3 STORE_FAST          3 (3)
          6 LOAD_GLOBAL         3 (3)
          9 LOAD_FAST           3 (3)
         12 CALL_FUNCTION       1
         15 PRINT_ITEM     
         16 PRINT_NEWLINE  
         17 LOAD_CONST          3 (3)
         20 LOAD_CONST          0 (0)
         23 IMPORT_NAME         4 (4)
         26 STORE_FAST          4 (4)
         29 LOAD_GLOBAL         0 (0)
         32 LOAD_ATTR           1 (1)
         35 CALL_FUNCTION       0
         38 LOAD_ATTR           5 (5)
         41 LOAD_ATTR           6 (6)
         44 LOAD_FAST           3 (3)
         47 SLICE+1        
         48 STORE_FAST          5 (5)
         51 LOAD_FAST           5 (5)
         54 LOAD_FAST           4 (4)
         57 LOAD_ATTR           4 (4)
         60 LOAD_FAST           5 (5)
         63 CALL_FUNCTION       1
         66 BUILD_TUPLE         2
         69 PRINT_ITEM     
         70 PRINT_NEWLINE  
         71 LOAD_GLOBAL         7 (7)

'g'

In [1]:
!touch tests/__init__.py

In [6]:
!python2 test.py

[FRAME_INIT] _symbolicInputNode, _symbolicOutputNode
[Dillable]
.
----------------------------------------------------------------------
Ran 1 test in 0.062s

OK


In [23]:
# lambda : raise Exception()

Overwriting test.py


In [40]:
a = object()
# a.__

object

In [ ]:
import dill
d = dill.loads(open('test_build/pipe.py.index','r').read())
d

In [18]:
!python2 test.py testCase.test_code_syno_change

[FRAME_INIT] _symbolicInputNode, _symbolicOutputNode
[indexFile] BaseFile('tests/example_string_short.py.index')
(0, 'tests-number.txt')
(1, 'tests-letter.txt')
[CMD]
    echo "1"> tests-number.txt; 
    echo a>tests-letter.txt; 
            

[FRAME_INIT] _symbolicInputNode, _symbolicOutputNode
[indexFile] BaseFile('pipe.py.index')
(0, 'tests-number.txt')
(1, 'tests-letter.txt')
START--------------------
CHECKING_NODE:<Node with func:_symbolicInputNode>
(<Node with func:_symbolicInputNode>, 1, ['INDEX_ABSENT'])
CHANGED_CODE:<Node with func:_symbolicInputNode>
RUNNING:<Node with func:_symbolicInputNode>
CHECKING_NODE:<Node with func:InputFileNode_0000:tag:tests-number.txt>
(<Node with func:InputFileNode_0000:tag:tests-number.txt>, 1, ['INDEX_ABSENT'])
CHANGED_CODE:<Node with func:InputFileNode_0000:tag:tests-number.txt>
RUNNING:<Node with func:InputFileNode_0000:tag:tests-number.txt>
CHECKING_NODE:<Node with func:InputFileNode_0001:tag:tests-letter.txt>
(<Node with func:InputFileNode_0

In [28]:
!python2 test.py testCase.test_code_syno_change

[FRAME_INIT] _symbolicInputNode, _symbolicOutputNode
[indexFile] BaseFile('tests/example_string_short.py.index')
(0, 'tests-number.txt')
(1, 'tests-letter.txt')
[CMD]
    echo "1"> tests-number.txt; 
    echo a>tests-letter.txt; 
            

[FRAME_INIT] _symbolicInputNode, _symbolicOutputNode
[indexFile] BaseFile('pipe.py.index')
(0, 'tests-number.txt')
(1, 'tests-letter.txt')
START--------------------
CHECKING_NODE:<Node with func:_symbolicInputNode>
(<Node with func:_symbolicInputNode>, 1, ['INDEX_ABSENT'])
CHANGED_CODE:<Node with func:_symbolicInputNode>
RUNNING:<Node with func:_symbolicInputNode>
CHECKING_NODE:<Node with func:InputFileNode_0000:tag:tests-number.txt>
(<Node with func:InputFileNode_0000:tag:tests-number.txt>, 1, ['INDEX_ABSENT'])
CHANGED_CODE:<Node with func:InputFileNode_0000:tag:tests-number.txt>
RUNNING:<Node with func:InputFileNode_0000:tag:tests-number.txt>
CHECKING_NODE:<Node with func:InputFileNode_0001:tag:tests-letter.txt>
(<Node with func:InputFileNode_0

In [854]:
x,y = '@Node.from_func({\n    "OUT":TrackedFile("tests-out10.txt"),\n})\ndef out10(  self, numberFile, letterFile ):\n    \'\'\'\n    some doc\n    \'\'\'\n    number = numberFile().open(\'r\').read().strip()\n    letter = letterFile().open(\'r\').read().strip()\n    with self.output_kw[\'OUT\'].open("w") as f:\n        f.write( 10 * (number+letter)+\'\\n\')\n    return\n', '@Node.from_func({\n    "OUT":TrackedFile("tests-out10.txt"),\n})\ndef out10(  self, numberFile, letterFile ):\n    \n    number = numberFile().open(\'r\').read().strip()\n    letter = letterFile().open(\'r\').read().strip()\n    with self.output_kw[\'OUT\'].open("w") as f:\n        f.write( 10 * (number+letter)+\'\\n\')\n    return\n'
print x
print y


@Node.from_func({
    "OUT":TrackedFile("tests-out10.txt"),
})
def out10(  self, numberFile, letterFile ):
    '''
    some doc
    '''
    number = numberFile().open('r').read().strip()
    letter = letterFile().open('r').read().strip()
    with self.output_kw['OUT'].open("w") as f:
        f.write( 10 * (number+letter)+'\n')
    return

@Node.from_func({
    "OUT":TrackedFile("tests-out10.txt"),
})
def out10(  self, numberFile, letterFile ):
    
    number = numberFile().open('r').read().strip()
    letter = letterFile().open('r').read().strip()
    with self.output_kw['OUT'].open("w") as f:
        f.write( 10 * (number+letter)+'\n')
    return

